Predspracovanie údajov pre strojové učenie

    2.1 Realizácia predspracovania dát

        (A-1b) Oddelenie testovacich a trénovacich údajov 

        (B-1b) Transformacia dát na vhodný formát pre ML

        (C-2b) Transformacia atribútov dát pre strojové učenie

        (D-1b) Všeobecné závery

    2.2 Výber atribútov pre strojové učenie

        (A-3b) Výpočet informatívnych atribútov pre výpočetanie premennej

        (B-1b) Zoradenie atribútov podľa dôležitosti

        (C-1b) Zdôvodnenie rozhodnuti pre realizáciu 

    2.3  Replikovateľnosť predspracovania

(A-3b) Upraveny kód realizujúci predspracovanie trénovacej množiny tak, aby ho bolo možné bez ďalších úprav znovu použiť na predspracovanie testovacej množiny v kontexte strojového učenia


(B-2b) Využitee možnosti sklearn.pipeline